In [1]:
import pandas as pd
from rdflib import Graph, RDF, Namespace

In [2]:
defects_map = pd.read_csv('defects_hpo_mappings.tsv', sep='\t', index_col=0)

In [3]:
g = Graph()

disease = Namespace("https://www.orpha.net/ORDO/")
phenotype = Namespace("https://purl.obolibrary.org/obo/")
relationship = Namespace("https://semanticscience.org/resource/")
gene_symbol = Namespace("https://identifiers.org/hgnc.symbol/")

In [4]:
g.bind('gene_symbol', gene_symbol)
g.bind('disease', disease)
g.bind('relationship', relationship)
g.bind('phenotype', phenotype)

In [5]:
with open('CDC-birth-defect-genes-Geneshot.gmt', 'r') as f: 
    for l in f.readlines(): 
        p = l.split('\t')[0].strip()
        hp_id = defects_map.loc[p, 'hp']
        if hp_id.startswith('ORPHA'):
            hp_id = hp_id.replace(':', '_').replace('ORPHA', 'Orphanet')
            g.add((disease[hp_id], RDF.type, relationship['SIO_010299']))
            for gene in l.split('\t')[2:]:
                g.add((disease[hp_id], relationship['SIO_000983'], gene_symbol[gene]))
        else:
            hp_id = hp_id.replace(':', '_')
            g.add((phenotype[hp_id], RDF.type, relationship['SIO_010056']))
            for gene in l.split('\t')[2:]:
                g.add((phenotype[hp_id], relationship['SIO_000983'], gene_symbol[gene.strip()]))

In [6]:
g.serialize(format="turtle", destination="Geneshot_BirthDefects_Gene_Associations.ttl")

<Graph identifier=Ndb86689e93744735a9e5fa86518689d9 (<class 'rdflib.graph.Graph'>)>